In [19]:
import torch
import torch.nn as nn

torch.cuda.set_device(0)
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())

# Print memory reserved and allocated
print(torch.cuda.memory_reserved())
print(torch.cuda.memory_allocated())

1.13.1+cu117
11.7
True
3179282432
1411846144


In [20]:
# Initialize your model
model = YOLO('yolov8m-seg.pt')

# Choose GPU 1 directly
device = 'cuda:1'

# Move model to GPU 1
model.to(device)

# To use DataParallel and both GPUs have enough memory
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    model = nn.DataParallel(model)

Using 2 GPUs!


In [5]:
pip install -r ~/Documents/inseer_ML_Project/yolov8/requirements.txt


Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.7/645.7 kB 16.6 MB/s eta 0:00:00m eta 0:00:01
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.0.215
    Uninstalling ultralytics-8.0.215:
      Successfully uninstalled ultralytics-8.0.215
Note: you may need to restart the kernel to use updated packages.


In [7]:
from ultralytics import YOLO
import cv2
from pathlib import Path
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
from types import SimpleNamespace
import time
from sklearn.model_selection import train_test_split
from zipfile import ZipFile
import shutil
import glob

In [16]:
# Paths to the images and labels
images_path = "/home/nikhil/Documents/Datasets/GTEA dataset/GTEA_original_croppeddataset/Cropped_Images"
labels_path = "/home/nikhil/Documents/Datasets/GTEA dataset/GTEA_original_croppeddataset/Cropped_yoloLabels"

In [61]:
# Function to split dataset into train and test
import os
import shutil
import random

# Set the seed for reproducibility
random.seed(42)

# Define the paths
source_images = "/home/nikhil/Documents/Datasets/GTEA dataset/GTEA_original_croppeddataset/Cropped_Images"
source_labels = "/home/nikhil/Documents/Datasets/GTEA dataset/GTEA_original_croppeddataset/Cropped_yoloLabels"
destination = "/home/nikhil/Documents/Datasets/GTEA dataset/GTEA_original_croppeddataset/GTEA_splitdata/"

# Create train and test directories
os.makedirs(destination + 'train/images', exist_ok=True)
os.makedirs(destination + 'train/labels', exist_ok=True)
os.makedirs(destination + 'test/images', exist_ok=True)
os.makedirs(destination + 'test/labels', exist_ok=True)

# List all images
all_images = [f for f in os.listdir(source_images) if os.path.isfile(os.path.join(source_images, f))]
# Shuffle the list
random.shuffle(all_images)

# Split images into train and test (90% train, 10% test)
split_idx = int(0.9 * len(all_images))
train_images = all_images[:split_idx]
test_images = all_images[split_idx:]

# Function to copy files
def copy_files(files, src_folder, dest_folder):
    for file in files:
        src_file = os.path.join(src_folder, file)
        dest_file = os.path.join(dest_folder, file)
        shutil.copy(src_file, dest_file)

# Copy train images and labels
copy_files(train_images, source_images, destination + 'train/images/')
copy_files([f.replace('.jpg', '.txt') for f in train_images], source_labels, destination + 'train/labels/')

# Copy test images and labels
copy_files(test_images, source_images, destination + 'test/images/')
copy_files([f.replace('.jpg', '.txt') for f in test_images], source_labels, destination + 'test/labels/')

print(f"Dataset split completed. Train set: {len(train_images)} images, Test set: {len(test_images)} images.")


Dataset split completed. Train set: 5960 images, Test set: 663 images.


In [62]:
# Create YAML configuration for training
def create_yaml_configuration(train_path, test_path, val_path, num_classes=1):
    yaml_content = f"""
# YoloV8 training on GTEA dataset
train: /home/nikhil/Documents/Datasets/GTEA dataset/GTEA_original_croppeddataset/GTEA_splitdata/train
test: /home/nikhil/Documents/Datasets/GTEA dataset/GTEA_original_croppeddataset/GTEA_splitdata/test
val: /home/nikhil/Documents/Datasets/GTEA dataset/GTEA_original_croppeddataset/GTEA_splitdata/test

# Classes
nc: {num_classes}
names: ['hand']
    """
    yaml_file_path = '/home/nikhil/Documents/Datasets/GTEA dataset/GTEA_original_croppeddataset/GTEA_splitdata/GTEA_crop_hands.yaml'  # Path where the YAML file will be saved
    with open(yaml_file_path, 'w') as yaml_file:
        yaml_file.write(yaml_content)
    return yaml_file_path

# Adjusting the paths for train, test, and val
train_path = os.path.join(images_path, 'train')
test_path = os.path.join(images_path, 'test')
val_path = os.path.join(images_path, 'test')  # Assuming validation and test paths are the same

yaml_file_path = create_yaml_configuration(train_path, test_path, val_path)


In [63]:
#training model
def train_model(yaml_file_path, batch=1, imgsz=640):
    model = YOLO("yolov8m-seg.pt")
    model.train(data=yaml_file_path, epochs=80, batch=batch, imgsz=imgsz, project="GTEA-Hand", name="GTEA_Hand_Segmentation")
    model.val()


train_model('/home/nikhil/Documents/Datasets/GTEA dataset/GTEA_original_croppeddataset/GTEA_splitdata/GTEA_crop_hands.yaml', batch=1)


New https://pypi.org/project/ultralytics/8.0.219 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.216 🚀 Python-3.9.7 torch-1.13.1+cu117 CUDA:0 (NVIDIA TITAN RTX, 24220MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=segment, mode=train, model=yolov8m-seg.pt, data=/home/nikhil/Documents/Datasets/GTEA dataset/GTEA_original_croppeddataset/GTEA_splitdata/GTEA_crop_hands.yaml, epochs=80, patience=50, batch=1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=GTEA-Hand, name=GTEA_Hand_Segmentation, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, sh

Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks failed ❌. Anomalies were detected with AMP on your system that may lead to NaN losses or zero-mAP results, so AMP will be disabled during training.


train: Scanning /home/nikhil/Documents/Datasets/GTEA dataset/GTEA_original_croppeddataset/GTEA_splitdata/train/labels... 5960 images, 799 backgrounds, 0 corrupt: 100%|██████████| 5960/5960 [00:10<00:00, 584.41it/s]


train: New cache created: /home/nikhil/Documents/Datasets/GTEA dataset/GTEA_original_croppeddataset/GTEA_splitdata/train/labels.cache


val: Scanning /home/nikhil/Documents/Datasets/GTEA dataset/GTEA_original_croppeddataset/GTEA_splitdata/test/labels... 663 images, 74 backgrounds, 0 corrupt: 100%|██████████| 663/663 [00:01<00:00, 600.54it/s]

...

                   all        663        589      0.981      0.956      0.985      0.907      0.993      0.956      0.986      0.879
Speed: 1.0ms preprocess, 10.0ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to GTEA-Hand/GTEA_Hand_Segmentation


lr/pg0,▃████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg1,▃████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg2,▃████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
metrics/mAP50(B),▃▁▄▅▃▇▆▆▇▆▆▇▆▇▇▇▆█▇▇▇▆▇▇▇█▇▇█▇▇▇▇██▇▇▆▆█
metrics/mAP50(M),▂▁▅▆▅▇▆▇▇▇▇█▇█▇█▇██▇█▇██████████████▇▇▇█
metrics/mAP50-95(B),▂▁▄▅▅▅▆▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇█▇███████████▇█
metrics/mAP50-95(M),▁▁▅▅▅▆▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇████████████████
metrics/precision(B),▂▁▂▄▁▃▄▆▆▆▆▅▃▆▆▆▅▇▆▆▇█▆▇▇██▇▇▇▆▆▆▇▆█▇▇█▆
metrics/precision(M),▃▁▄▅▄▄▄▆▆▆▆▅▅▇▇▇▆▇▇▇██▇▇▇▇█▇██▇▇▇███▇▇██
metrics/recall(B),▁▁▂▂▁▇▆▄▄▄▄▇▃▆▄▇▅▇▇▆▆▃▆▇▆▅▆▇▇▇▇▇▆█▇▅▆▄▄▇
metrics/recall(M),▁▁▃▄▂▅▆▅▅▆▅▇▅▇▅▇▆▇▇▇▆▅▇█▇▇▇▇▇▇██▇█▇▆▇▆▆▇


Ultralytics YOLOv8.0.216 🚀 Python-3.9.7 torch-1.13.1+cu117 CUDA:0 (NVIDIA TITAN RTX, 24220MiB)
YOLOv8m-seg summary (fused): 245 layers, 27222963 parameters, 0 gradients, 110.0 GFLOPs


val: Scanning /home/nikhil/Documents/Datasets/GTEA dataset/GTEA_original_croppeddataset/GTEA_splitdata/test/labels.cache... 663 images, 74 backgrounds, 0 corrupt: 100%|██████████| 663/663 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 663/663 [00:29<00:00, 22.70it/s]


                   all        663        589      0.981      0.956      0.986      0.907      0.993      0.956      0.986       0.88
Speed: 1.6ms preprocess, 24.0ms inference, 0.0ms loss, 3.8ms postprocess per image
Results saved to GTEA-Hand/GTEA_Hand_Segmentation2


In [64]:
#Testing model
import os
import cv2
import numpy as np
from glob import glob
from pathlib import Path
from random import randint
import matplotlib as mpl
from shapely.geometry import Polygon
import matplotlib.pyplot as plt

def plot_polygon_on_image(image, polygon):
    polygon = np.asarray(polygon)
    polygon = np.squeeze(polygon)
    image = cv2.polylines(image, [polygon.astype('int')], True, (randint(0, 255), randint(0, 255), randint(0, 255)),
                  2)  # Draw Poly Lines
    image = cv2.fillPoly(image, [polygon.astype('int')], (randint(0,255),randint(0,255),randint(0,255)), cv2.LINE_AA) # Draw area

    return image

def test_model(weights_path, image_dir, show=False):
    image_dir = Path(image_dir)
    save_dir = Path(image_dir).parent / "test_yolov8_segs"
    save_dir.mkdir(exist_ok=True, parents=True)
    assert image_dir.is_dir()
    # Load a model
    model = YOLO(weights_path)  # load pretrained segmentation model
    # predict with model
    for image_path in image_dir.iterdir():
        image_ext = image_path.suffix
        print("image name", image_path.name)
        if image_ext == ".npy":
            image = np.load(image_path.as_posix(), allow_pickle=True)
        else:
            image = cv2.imread(image_path.as_posix())

        results = model.predict(source=image, conf=0.5, iou=0.4)
        seg = results[0]
        if seg:
            all_segs = []
            for i in range(len(seg.masks.xy)):
                seg_xy = seg.masks.xy[i]
                all_segs.append(seg_xy)
                image = plot_polygon_on_image(image, seg_xy)
            seg_save_path = save_dir / image_path.with_suffix('.npy').name
            np.save(seg_save_path.as_posix(), all_segs, allow_pickle=True)
            if show:
                plt.imshow(image)
                plt.show()
        else:
            print(f"no results for image {image_path.name}!")



if __name__ == "__main__":
    weights_path = "/home/nikhil/Documents/inseer_ML_Project/yolov8/Nikhil-yolov8 training/GTEAtraining_yolo/GTEA-Hand/GTEA_Hand_Segmentation/weights/best.pt"
    image_dir = "/home/nikhil/Documents/Datasets/GTEA dataset/GTEA_original_croppeddataset/GTEA_splitdata/test/images"

    test_model(weights_path, image_dir, show=True)

image name P13-R01-PastaSalad_000536.jpg

0: 608x640 1 hand, 35.9ms
Speed: 6.5ms preprocess, 35.9ms inference, 1.5ms postprocess per image at shape (1, 3, 608, 640)
image name P01-R02-TurkeySandwich_000067.jpg

0: 640x640 (no detections), 37.0ms
Speed: 6.1ms preprocess, 37.0ms inference, 4.7ms postprocess per image at shape (1, 3, 640, 640)
no results for image P01-R02-TurkeySandwich_000067.jpg!
image name P02-R05-Cheeseburger_000126.jpg

0: 640x640 1 hand, 34.2ms
Speed: 4.7ms preprocess, 34.2ms inference, 21.1ms postprocess per image at shape (1, 3, 640, 640)
image name P22-R03-BaconAndEggs_000090.jpg



/tmp/ipykernel_1682225/4187078736.py:49: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


0: 640x576 1 hand, 35.6ms
Speed: 3.1ms preprocess, 35.6ms inference, 17.2ms postprocess per image at shape (1, 3, 640, 576)
image name P24-R03-BaconAndEggs_000170.jpg

0: 640x640 1 hand, 36.0ms
Speed: 3.9ms preprocess, 36.0ms inference, 16.6ms postprocess per image at shape (1, 3, 640, 640)
image name P26-R05-Cheeseburger_000319.jpg

0: 640x640 1 hand, 29.2ms
Speed: 2.3ms preprocess, 29.2ms inference, 14.9ms postprocess per image at shape (1, 3, 640, 640)
image name P06-R01-PastaSalad_000150.jpg

0: 640x640 1 hand, 30.0ms
Speed: 2.9ms preprocess, 30.0ms inference, 21.0ms postprocess per image at shape (1, 3, 640, 640)
image name OP05-R04-ContinentalBreakfast_003558.jpg

0: 640x640 1 hand, 24.0ms
Speed: 3.5ms preprocess, 24.0ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)
image name P19-R03-BaconAndEggs_000170.jpg

0: 576x640 1 hand, 26.1ms
Speed: 3.8ms preprocess, 26.1ms inference, 1.6ms postprocess per image at shape (1, 3, 576, 640)
image name P02-R03-BaconAndEggs

image name P02-R03-BaconAndEggs_000282.jpg

0: 640x608 1 hand, 20.8ms
Speed: 3.1ms preprocess, 20.8ms inference, 9.5ms postprocess per image at shape (1, 3, 640, 608)
image name OP03-R06-GreekSalad_000124.jpg

0: 640x608 1 hand, 29.9ms
Speed: 3.3ms preprocess, 29.9ms inference, 17.0ms postprocess per image at shape (1, 3, 640, 608)
image name P21-R03-BaconAndEggs_000027.jpg

0: 416x640 1 hand, 30.0ms
Speed: 1.4ms preprocess, 30.0ms inference, 21.1ms postprocess per image at shape (1, 3, 416, 640)
image name P16-R03-BaconAndEggs_000358.jpg

0: 640x608 1 hand, 37.7ms
Speed: 3.9ms preprocess, 37.7ms inference, 17.1ms postprocess per image at shape (1, 3, 640, 608)
image name P21-R03-BaconAndEggs_000082.jpg

0: 640x640 (no detections), 36.7ms
Speed: 5.1ms preprocess, 36.7ms inference, 3.7ms postprocess per image at shape (1, 3, 640, 640)
no results for image P21-R03-BaconAndEggs_000082.jpg!
image name P02-R05-Cheeseburger_000039.jpg

0: 480x640 1 hand, 29.0ms
Speed: 2.5ms preprocess, 29.0m

image name P20-R03-BaconAndEggs_000195.jpg

0: 640x640 (no detections), 34.3ms
Speed: 2.5ms preprocess, 34.3ms inference, 4.4ms postprocess per image at shape (1, 3, 640, 640)
no results for image P20-R03-BaconAndEggs_000195.jpg!
image name P18-R04-ContinentalBreakfast_000033.jpg

0: 544x640 1 hand, 36.6ms
Speed: 5.2ms preprocess, 36.6ms inference, 17.2ms postprocess per image at shape (1, 3, 544, 640)
image name OP01-R02-TurkeySandwich_000064.jpg

0: 608x640 1 hand, 35.5ms
Speed: 2.4ms preprocess, 35.5ms inference, 16.8ms postprocess per image at shape (1, 3, 608, 640)
image name P02-R05-Cheeseburger_000118.jpg

0: 480x640 1 hand, 29.8ms
Speed: 1.6ms preprocess, 29.8ms inference, 11.3ms postprocess per image at shape (1, 3, 480, 640)
image name P17-R03-BaconAndEggs_000231.jpg

0: 544x640 1 hand, 33.3ms
Speed: 2.4ms preprocess, 33.3ms inference, 20.9ms postprocess per image at shape (1, 3, 544, 640)
image name P10-R01-PastaSalad_000153.jpg

0: 576x640 1 hand, 18.3ms
Speed: 1.7ms prepro

...

image name P10-R01-PastaSalad_000019.jpg

0: 640x576 1 hand, 35.2ms
Speed: 4.1ms preprocess, 35.2ms inference, 8.4ms postprocess per image at shape (1, 3, 640, 576)
image name P02-R01-PastaSalad_000072.jpg

0: 640x640 1 hand, 40.9ms
Speed: 3.6ms preprocess, 40.9ms inference, 21.2ms postprocess per image at shape (1, 3, 640, 640)
image name OP06-R07-Pizza_006888.jpg

0: 576x640 1 hand, 33.7ms
Speed: 3.2ms preprocess, 33.7ms inference, 17.5ms postprocess per image at shape (1, 3, 576, 640)
image name P03-R01-PastaSalad_000433.jpg

0: 640x640 1 hand, 36.8ms
Speed: 3.6ms preprocess, 36.8ms inference, 17.4ms postprocess per image at shape (1, 3, 640, 640)
image name OP01-R07-Pizza_000152.jpg

0: 640x640 1 hand, 28.9ms
Speed: 3.2ms preprocess, 28.9ms inference, 21.0ms postprocess per image at shape (1, 3, 640, 640)
image name OP03-R07-Pizza_003243.jpg

0: 640x640 1 hand, 27.9ms
Speed: 4.3ms preprocess, 27.9ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)
image name OP03-R0

image name P08-R01-PastaSalad_000302.jpg

0: 640x640 (no detections), 36.5ms
Speed: 4.7ms preprocess, 36.5ms inference, 4.7ms postprocess per image at shape (1, 3, 640, 640)
no results for image P08-R01-PastaSalad_000302.jpg!
image name P21-R05-Cheeseburger_000025.jpg

0: 640x640 (no detections), 32.4ms
Speed: 5.1ms preprocess, 32.4ms inference, 3.8ms postprocess per image at shape (1, 3, 640, 640)
no results for image P21-R05-Cheeseburger_000025.jpg!
image name P06-R02-TurkeySandwich_000092.jpg

0: 640x640 (no detections), 32.2ms
Speed: 4.1ms preprocess, 32.2ms inference, 4.5ms postprocess per image at shape (1, 3, 640, 640)
no results for image P06-R02-TurkeySandwich_000092.jpg!
image name P19-R03-BaconAndEggs_000293.jpg

0: 640x640 1 hand, 33.6ms
Speed: 4.6ms preprocess, 33.6ms inference, 21.1ms postprocess per image at shape (1, 3, 640, 640)
image name P04-R01-PastaSalad_000207.jpg

0: 512x640 (no detections), 28.4ms
Speed: 3.2ms preprocess, 28.4ms inference, 2.6ms postprocess per 

no results for image P22-R03-BaconAndEggs_000103.jpg!
image name P01-R02-TurkeySandwich_000068.jpg

0: 544x640 1 hand, 29.5ms
Speed: 3.8ms preprocess, 29.5ms inference, 21.1ms postprocess per image at shape (1, 3, 544, 640)
image name P19-R03-BaconAndEggs_000071.jpg

0: 576x640 1 hand, 35.2ms
Speed: 4.1ms preprocess, 35.2ms inference, 16.5ms postprocess per image at shape (1, 3, 576, 640)
image name OP02-R03-BaconAndEggs_012703.jpg

0: 544x640 1 hand, 34.7ms
Speed: 5.0ms preprocess, 34.7ms inference, 16.4ms postprocess per image at shape (1, 3, 544, 640)
image name P07-R01-PastaSalad_000327.jpg

0: 640x640 1 hand, 35.8ms
Speed: 3.3ms preprocess, 35.8ms inference, 3.4ms postprocess per image at shape (1, 3, 640, 640)
image name OP06-R07-Pizza_012759.jpg

0: 640x640 1 hand, 31.1ms
Speed: 4.5ms preprocess, 31.1ms inference, 21.3ms postprocess per image at shape (1, 3, 640, 640)
image name P24-R03-BaconAndEggs_000174.jpg

0: 640x640 1 hand, 24.7ms
Speed: 4.3ms preprocess, 24.7ms inference,

0: 640x640 1 hand, 36.5ms
Speed: 3.2ms preprocess, 36.5ms inference, 21.1ms postprocess per image at shape (1, 3, 640, 640)
image name P10-R05-Cheeseburger_000025.jpg

0: 608x640 1 hand, 36.7ms
Speed: 3.5ms preprocess, 36.7ms inference, 9.3ms postprocess per image at shape (1, 3, 608, 640)
image name P13-R01-PastaSalad_000218.jpg

0: 640x640 1 hand, 23.4ms
Speed: 3.3ms preprocess, 23.4ms inference, 5.6ms postprocess per image at shape (1, 3, 640, 640)
image name P20-R04-ContinentalBreakfast_000126.jpg

0: 480x640 1 hand, 29.8ms
Speed: 2.7ms preprocess, 29.8ms inference, 13.8ms postprocess per image at shape (1, 3, 480, 640)
image name P19-R03-BaconAndEggs_000106.jpg

0: 640x608 (no detections), 34.3ms
Speed: 2.0ms preprocess, 34.3ms inference, 4.7ms postprocess per image at shape (1, 3, 640, 608)
no results for image P19-R03-BaconAndEggs_000106.jpg!
image name OP04-R06-GreekSalad_000036.jpg

0: 608x640 1 hand, 37.8ms
Speed: 2.7ms preprocess, 37.8ms inference, 21.0ms postprocess per ima

0: 608x640 1 hand, 26.8ms
Speed: 2.3ms preprocess, 26.8ms inference, 11.0ms postprocess per image at shape (1, 3, 608, 640)
image name P16-R03-BaconAndEggs_000422.jpg

0: 640x640 1 hand, 37.4ms
Speed: 4.5ms preprocess, 37.4ms inference, 21.2ms postprocess per image at shape (1, 3, 640, 640)
image name P26-R05-Cheeseburger_000309.jpg

0: 640x640 (no detections), 34.2ms
Speed: 2.1ms preprocess, 34.2ms inference, 4.5ms postprocess per image at shape (1, 3, 640, 640)
no results for image P26-R05-Cheeseburger_000309.jpg!
image name OP04-R03-BaconAndEggs_002325.jpg

0: 640x640 (no detections), 34.0ms
Speed: 4.4ms preprocess, 34.0ms inference, 4.6ms postprocess per image at shape (1, 3, 640, 640)
no results for image OP04-R03-BaconAndEggs_002325.jpg!
image name P10-R02-TurkeySandwich_000083.jpg

0: 608x640 1 hand, 33.9ms
Speed: 2.9ms preprocess, 33.9ms inference, 17.4ms postprocess per image at shape (1, 3, 608, 640)
image name OP04-R06-GreekSalad_000065.jpg

0: 640x640 1 hand, 37.3ms
Speed: 

In [65]:
#Code to plot the segments(in .npy format) on the images
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import cv2

def display_segments_on_images(image_dir, segmentation_dir, save_dir):
    image_dir = Path(image_dir)
    segmentation_dir = Path(segmentation_dir)
    save_dir = Path(save_dir)
    save_dir.mkdir(exist_ok=True)  # Ensure the save directory exists
    
    # Iterate over .npy files and display them on the corresponding images
    for seg_file in segmentation_dir.glob('*.npy'):
        image_file = image_dir / (seg_file.stem + '.jpg')
        if not image_file.exists():
            print(f"No corresponding image for segmentation {seg_file.stem}")
            continue
        
        image = cv2.imread(str(image_file))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
        
        segmentations = np.load(seg_file, allow_pickle=True)
        
        plt.figure()
        plt.imshow(image)
        
        for polygon in segmentations:
            plt.plot(*np.array(polygon).T, marker='o', color='red')
        
        plt.axis('off')
        save_path = save_dir / f"{image_file.stem}_segmented.png"
        plt.savefig(save_path, bbox_inches='tight', pad_inches=0)
        plt.close()

# Define the paths to directories
image_dir = "/home/nikhil/Documents/Datasets/GTEA dataset/GTEA_original_croppeddataset/GTEA_splitdata/test/images"


# Path to the directory where the .npy segmentations are stored
segs_dir = '/home/nikhil/Documents/Datasets/GTEA dataset/GTEA_original_croppeddataset/GTEA_splitdata/test/test_yolov8_segs'
# Directory to save the plots
plots_save_dir = '/home/nikhil/Documents/Datasets/GTEA dataset/GTEA_original_croppeddataset/GTEA_splitdata/test/saved_plots'

display_segments_on_images(images_dir, segs_dir, plots_save_dir)


In [66]:
#Testing model with 0.3 confidence
import os
import cv2
import numpy as np
from glob import glob
from pathlib import Path
from random import randint
import matplotlib as mpl
from shapely.geometry import Polygon
import matplotlib.pyplot as plt

def plot_polygon_on_image(image, polygon):
    polygon = np.asarray(polygon)
    polygon = np.squeeze(polygon)
    image = cv2.polylines(image, [polygon.astype('int')], True, (randint(0, 255), randint(0, 255), randint(0, 255)),
                  2)  # Draw Poly Lines
    image = cv2.fillPoly(image, [polygon.astype('int')], (randint(0,255),randint(0,255),randint(0,255)), cv2.LINE_AA) # Draw area

    return image

def test_model(weights_path, image_dir, show=False):
    image_dir = Path(image_dir)
    save_dir = Path(image_dir).parent / "test_yolov8_segs_0.3conf"
    save_dir.mkdir(exist_ok=True, parents=True)
    assert image_dir.is_dir()
    # Load a model
    model = YOLO(weights_path)  # load pretrained segmentation model
    # predict with model
    for image_path in image_dir.iterdir():
        image_ext = image_path.suffix
        print("image name", image_path.name)
        if image_ext == ".npy":
            image = np.load(image_path.as_posix(), allow_pickle=True)
        else:
            image = cv2.imread(image_path.as_posix())

        results = model.predict(source=image, conf=0.3, iou=0.4)
        seg = results[0]
        if seg:
            all_segs = []
            for i in range(len(seg.masks.xy)):
                seg_xy = seg.masks.xy[i]
                all_segs.append(seg_xy)
                image = plot_polygon_on_image(image, seg_xy)
            seg_save_path = save_dir / image_path.with_suffix('.npy').name
            np.save(seg_save_path.as_posix(), all_segs, allow_pickle=True)
            if show:
                plt.imshow(image)
                plt.show()
        else:
            print(f"no results for image {image_path.name}!")



if __name__ == "__main__":
    weights_path = "/home/nikhil/Documents/inseer_ML_Project/yolov8/Nikhil-yolov8 training/GTEAtraining_yolo/GTEA-Hand/GTEA_Hand_Segmentation/weights/best.pt"
    image_dir = "/home/nikhil/Documents/Datasets/GTEA dataset/GTEA_original_croppeddataset/GTEA_splitdata/test/images"
    test_model(weights_path, image_dir, show=True)

image name P13-R01-PastaSalad_000536.jpg

0: 608x640 1 hand, 36.8ms
Speed: 9.0ms preprocess, 36.8ms inference, 17.6ms postprocess per image at shape (1, 3, 608, 640)
image name P01-R02-TurkeySandwich_000067.jpg

0: 640x640 (no detections), 36.7ms
Speed: 5.9ms preprocess, 36.7ms inference, 4.7ms postprocess per image at shape (1, 3, 640, 640)
no results for image P01-R02-TurkeySandwich_000067.jpg!
image name P02-R05-Cheeseburger_000126.jpg

0: 640x640 1 hand, 30.3ms
Speed: 3.2ms preprocess, 30.3ms inference, 19.2ms postprocess per image at shape (1, 3, 640, 640)
image name P22-R03-BaconAndEggs_000090.jpg

0: 640x576 1 hand, 20.2ms
Speed: 2.2ms preprocess, 20.2ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 576)
image name P24-R03-BaconAndEggs_000170.jpg



/tmp/ipykernel_1682225/3193695749.py:49: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


0: 640x640 1 hand, 36.9ms
Speed: 5.1ms preprocess, 36.9ms inference, 16.9ms postprocess per image at shape (1, 3, 640, 640)
image name P26-R05-Cheeseburger_000319.jpg

0: 640x640 1 hand, 34.2ms
Speed: 4.2ms preprocess, 34.2ms inference, 21.1ms postprocess per image at shape (1, 3, 640, 640)
image name P06-R01-PastaSalad_000150.jpg

0: 640x640 1 hand, 32.1ms
Speed: 5.7ms preprocess, 32.1ms inference, 16.9ms postprocess per image at shape (1, 3, 640, 640)
image name OP05-R04-ContinentalBreakfast_003558.jpg

0: 640x640 1 hand, 29.8ms
Speed: 3.5ms preprocess, 29.8ms inference, 17.3ms postprocess per image at shape (1, 3, 640, 640)
image name P19-R03-BaconAndEggs_000170.jpg

0: 576x640 1 hand, 35.3ms
Speed: 3.9ms preprocess, 35.3ms inference, 8.2ms postprocess per image at shape (1, 3, 576, 640)
image name P02-R03-BaconAndEggs_000184.jpg

0: 640x544 1 hand, 36.1ms
Speed: 3.7ms preprocess, 36.1ms inference, 20.9ms postprocess per image at shape (1, 3, 640, 544)
image name P02-R01-PastaSalad_

image name P21-R03-BaconAndEggs_000027.jpg

0: 416x640 1 hand, 24.7ms
Speed: 3.1ms preprocess, 24.7ms inference, 17.5ms postprocess per image at shape (1, 3, 416, 640)
image name P16-R03-BaconAndEggs_000358.jpg

0: 640x608 1 hand, 29.9ms
Speed: 2.0ms preprocess, 29.9ms inference, 7.5ms postprocess per image at shape (1, 3, 640, 608)
image name P21-R03-BaconAndEggs_000082.jpg

0: 640x640 (no detections), 34.9ms
Speed: 1.8ms preprocess, 34.9ms inference, 4.7ms postprocess per image at shape (1, 3, 640, 640)
no results for image P21-R03-BaconAndEggs_000082.jpg!
image name P02-R05-Cheeseburger_000039.jpg

0: 480x640 1 hand, 32.1ms
Speed: 2.3ms preprocess, 32.1ms inference, 21.0ms postprocess per image at shape (1, 3, 480, 640)
image name P07-R01-PastaSalad_000428.jpg

0: 640x640 (no detections), 30.0ms
Speed: 3.9ms preprocess, 30.0ms inference, 2.8ms postprocess per image at shape (1, 3, 640, 640)
no results for image P07-R01-PastaSalad_000428.jpg!
image name P11-R01-PastaSalad_000031.jpg


image name P02-R05-Cheeseburger_000118.jpg

0: 480x640 1 hand, 30.5ms
Speed: 3.2ms preprocess, 30.5ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)
image name P17-R03-BaconAndEggs_000231.jpg

0: 544x640 1 hand, 13.7ms
Speed: 1.9ms preprocess, 13.7ms inference, 4.0ms postprocess per image at shape (1, 3, 544, 640)
image name P10-R01-PastaSalad_000153.jpg

0: 576x640 1 hand, 31.7ms
Speed: 4.1ms preprocess, 31.7ms inference, 14.5ms postprocess per image at shape (1, 3, 576, 640)
image name P03-R01-PastaSalad_000400.jpg

0: 480x640 (no detections), 26.4ms
Speed: 4.1ms preprocess, 26.4ms inference, 4.6ms postprocess per image at shape (1, 3, 480, 640)
no results for image P03-R01-PastaSalad_000400.jpg!
image name OP05-R07-Pizza_000066.jpg

0: 640x640 1 hand, 38.1ms
Speed: 3.0ms preprocess, 38.1ms inference, 21.1ms postprocess per image at shape (1, 3, 640, 640)
image name P15-R01-PastaSalad_000064.jpg

0: 512x640 1 hand, 29.3ms
Speed: 6.5ms preprocess, 29.3ms inference, 1

...

no results for image P10-R05-Cheeseburger_000144.jpg!
image name P09-R01-PastaSalad_000266.jpg

0: 640x640 (no detections), 31.8ms
Speed: 3.5ms preprocess, 31.8ms inference, 2.7ms postprocess per image at shape (1, 3, 640, 640)
no results for image P09-R01-PastaSalad_000266.jpg!
image name OP04-R04-ContinentalBreakfast_001768.jpg

0: 640x640 (no detections), 29.5ms
Speed: 2.7ms preprocess, 29.5ms inference, 4.7ms postprocess per image at shape (1, 3, 640, 640)
no results for image OP04-R04-ContinentalBreakfast_001768.jpg!
image name P03-R01-PastaSalad_000295.jpg

0: 640x640 (no detections), 33.6ms
Speed: 6.4ms preprocess, 33.6ms inference, 4.6ms postprocess per image at shape (1, 3, 640, 640)
no results for image P03-R01-PastaSalad_000295.jpg!
image name P11-R01-PastaSalad_000434.jpg

0: 544x640 1 hand, 31.5ms
Speed: 4.2ms preprocess, 31.5ms inference, 1.3ms postprocess per image at shape (1, 3, 544, 640)
image name OP04-R06-GreekSalad_000067.jpg

0: 640x640 1 hand, 32.8ms
Speed: 3.8ms

image name P23-R03-BaconAndEggs_000187.jpg

0: 640x640 1 hand, 31.5ms
Speed: 3.8ms preprocess, 31.5ms inference, 16.4ms postprocess per image at shape (1, 3, 640, 640)
image name P10-R05-Cheeseburger_000050.jpg

0: 608x640 (no detections), 33.4ms
Speed: 1.9ms preprocess, 33.4ms inference, 4.7ms postprocess per image at shape (1, 3, 608, 640)
no results for image P10-R05-Cheeseburger_000050.jpg!
image name OP01-R01-PastaSalad_000068.jpg

0: 416x640 1 hand, 28.8ms
Speed: 2.8ms preprocess, 28.8ms inference, 21.1ms postprocess per image at shape (1, 3, 416, 640)
image name P10-R01-PastaSalad_000147.jpg

0: 608x640 1 hand, 17.3ms
Speed: 1.9ms preprocess, 17.3ms inference, 7.2ms postprocess per image at shape (1, 3, 608, 640)
image name OP03-R07-Pizza_002187.jpg

0: 608x640 1 hand, 29.7ms
Speed: 2.6ms preprocess, 29.7ms inference, 9.8ms postprocess per image at shape (1, 3, 608, 640)
image name P02-R01-PastaSalad_000071.jpg

0: 640x640 1 hand, 34.3ms
Speed: 4.8ms preprocess, 34.3ms inference

no results for image P16-R03-BaconAndEggs_000152.jpg!
image name P18-R03-BaconAndEggs_000240.jpg

0: 640x640 (no detections), 40.5ms
Speed: 2.2ms preprocess, 40.5ms inference, 4.6ms postprocess per image at shape (1, 3, 640, 640)
no results for image P18-R03-BaconAndEggs_000240.jpg!
image name P15-R01-PastaSalad_000317.jpg

0: 512x640 1 hand, 32.2ms
Speed: 4.2ms preprocess, 32.2ms inference, 12.6ms postprocess per image at shape (1, 3, 512, 640)
image name P01-R02-TurkeySandwich_000015.jpg

0: 640x640 1 hand, 21.7ms
Speed: 4.3ms preprocess, 21.7ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)
image name P15-R01-PastaSalad_000186.jpg

0: 640x512 (no detections), 28.8ms
Speed: 1.5ms preprocess, 28.8ms inference, 4.2ms postprocess per image at shape (1, 3, 640, 512)
no results for image P15-R01-PastaSalad_000186.jpg!
image name OP01-R07-Pizza_000138.jpg

0: 640x640 1 hand, 40.1ms
Speed: 3.2ms preprocess, 40.1ms inference, 16.6ms postprocess per image at shape (1, 3, 640

image name P09-R01-PastaSalad_000295.jpg

0: 640x640 1 hand, 20.8ms
Speed: 4.1ms preprocess, 20.8ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)
image name P02-R01-PastaSalad_000306.jpg

0: 640x640 1 hand, 20.4ms
Speed: 2.3ms preprocess, 20.4ms inference, 6.1ms postprocess per image at shape (1, 3, 640, 640)
image name P21-R05-Cheeseburger_000031.jpg

0: 448x640 1 hand, 25.0ms
Speed: 2.6ms preprocess, 25.0ms inference, 15.3ms postprocess per image at shape (1, 3, 448, 640)
image name P01-R02-TurkeySandwich_000077.jpg

0: 512x640 1 hand, 32.4ms
Speed: 2.6ms preprocess, 32.4ms inference, 1.7ms postprocess per image at shape (1, 3, 512, 640)
image name P18-R03-BaconAndEggs_000206.jpg

0: 512x640 (no detections), 26.0ms
Speed: 5.0ms preprocess, 26.0ms inference, 4.7ms postprocess per image at shape (1, 3, 512, 640)
no results for image P18-R03-BaconAndEggs_000206.jpg!
image name P23-R03-BaconAndEggs_000458.jpg

0: 640x640 (no detections), 40.7ms
Speed: 3.3ms preprocess,

image name P10-R06-GreekSalad_000060.jpg

0: 640x640 1 hand, 40.5ms
Speed: 2.1ms preprocess, 40.5ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)
image name OP01-R03-BaconAndEggs_000145.jpg

0: 640x608 1 hand, 15.4ms
Speed: 2.1ms preprocess, 15.4ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 608)
image name OP06-R04-ContinentalBreakfast_000061.jpg

0: 480x640 1 hand, 29.7ms
Speed: 3.3ms preprocess, 29.7ms inference, 12.9ms postprocess per image at shape (1, 3, 480, 640)
image name P21-R03-BaconAndEggs_000292.jpg

0: 448x640 1 hand, 31.7ms
Speed: 2.1ms preprocess, 31.7ms inference, 2.5ms postprocess per image at shape (1, 3, 448, 640)
image name OP02-R07-Pizza_000058.jpg

0: 640x640 1 hand, 40.4ms
Speed: 2.8ms preprocess, 40.4ms inference, 21.1ms postprocess per image at shape (1, 3, 640, 640)
image name P22-R03-BaconAndEggs_000309.jpg

0: 640x640 1 hand, 30.7ms
Speed: 2.0ms preprocess, 30.7ms inference, 17.0ms postprocess per image at shape (1, 3, 640

In [67]:
#Code to plot the segments(in .npy format) on the images 0.3 confidence
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import cv2

def display_segments_on_images(image_dir, segmentation_dir, save_dir):
    image_dir = Path(image_dir)
    segmentation_dir = Path(segmentation_dir)
    save_dir = Path(save_dir)
    save_dir.mkdir(exist_ok=True)  # Ensure the save directory exists
    
    # Iterate over .npy files and display them on the corresponding images
    for seg_file in segmentation_dir.glob('*.npy'):
        image_file = image_dir / (seg_file.stem + '.jpg')
        if not image_file.exists():
            print(f"No corresponding image for segmentation {seg_file.stem}")
            continue
        
        image = cv2.imread(str(image_file))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
        
        segmentations = np.load(seg_file, allow_pickle=True)
        
        plt.figure()
        plt.imshow(image)
        
        for polygon in segmentations:
            plt.plot(*np.array(polygon).T, marker='o', color='red')
        
        plt.axis('off')
        save_path = save_dir / f"{image_file.stem}_segmented.png"
        plt.savefig(save_path, bbox_inches='tight', pad_inches=0)
        plt.close()

image_dir = "/home/nikhil/Documents/Datasets/GTEA dataset/GTEA_original_croppeddataset/GTEA_splitdata/test/images"


# Path to the directory where the .npy segmentations are stored
segs_dir = '/home/nikhil/Documents/Datasets/GTEA dataset/GTEA_original_croppeddataset/GTEA_splitdata/test/test_yolov8_segs_0.3conf'
# Directory to save the plots
plots_save_dir = '/home/nikhil/Documents/Datasets/GTEA dataset/GTEA_original_croppeddataset/GTEA_splitdata/test/saved_plots_0.3conf'

display_segments_on_images(images_dir, segs_dir, plots_save_dir)



In [68]:
#To count number of empty labels in test dataset (no hands in images)
from pathlib import Path

def count_empty_files(directory):
    # Convert string path to Path object
    path = Path(directory)

    # Check if directory exists
    if not path.exists() or not path.is_dir():
        return "Directory does not exist or is not a directory."

    # Count empty files
    empty_files = sum(1 for file in path.glob('*.txt') if not file.stat().st_size)
    return empty_files

# Path to the labels directory
labels_directory = "/home/nikhil/Documents/Datasets/GTEA dataset/GTEA_original_croppeddataset/GTEA_splitdata/test/labels"

# Call the function and print the result
empty_file_count = count_empty_files(labels_directory)
print(f"Number of empty txt files: {empty_file_count}")


Number of empty txt files: 74
